In [168]:
import neural_network as nn
import definitions as d
import numpy as np
import os
from pandas import read_csv
import geopy.distance
from sklearn.preprocessing import OneHotEncoder
from collections import defaultdict
import datetime

def load_raw_data(path, type):
    files = os.listdir(f"{path}/{type}")
    data = {}
    surfix = f"_{type}.csv"
    for file in files:
        name = file[:file.find(surfix)]
        df = read_csv(f"{path}/{type}/{file}", sep=";")
        data[name] = df
    return data

def get_nearest_cittes(city_attributes):
    nerast_cities = {}
    for _, row in city_attributes.iterrows():
        source = (row["Latitude"], row["Longitude"])
        city_dist = []
        for _, row2 in city_attributes.iterrows():
            if row["City"] is row2["City"]:
                continue
            destination = (row2["Latitude"], row2["Longitude"])
            city_dist.append((row2["City"], geopy.distance.geodesic(source, destination).km))
        city_dist.sort(key=lambda x: x[1])
        nerast_cities[row["City"]] = [cd[0] for cd in city_dist[:3]]
    return nerast_cities
    
def load_train():
    dict = load_raw_data("data", "train")
    dict.pop("weather_description")
    for key, df in dict.items():
        dict[key] = df.iloc[12:, :]
    return dict

def load_test():
    dict = load_raw_data("data", "test")
    dict.pop("weather_description")
    for key, df in dict.items():
        dict[key] = df.iloc[:-1, :]
    return dict

def get_normalization_params(raw):
    params = {}
    for key, df in raw.items():
        all = np.reshape(df.to_numpy()[:, 1:], -1)
        params[key] = (np.nanmean(all), np.nanstd(all))
    return params

def to_city_time_vect(raw):
    cities = next(iter(raw.values())).columns[1:]
    hours = next(iter(raw.values()))[["datetime"]]
    city_time_dir = {c: hours.copy() for c in cities}
    for city in cities:
        for key, df in raw.items():
            city_time_dir[city][key] = df[[city]]
    return city_time_dir

def normalize(ctv, params):
    for df in ctv.values():
        for param, ms in params.items():
            mean, std = ms
            df[param] = (df[param] - mean) / std 

def normalize_city_attributes(city_attributes):
    la_mean = city_attributes["Latitude"].mean()
    la_std = city_attributes["Latitude"].std()
    city_attributes["Latitude"] = (city_attributes["Latitude"] - la_mean) / la_std
    lo_mean = city_attributes["Longitude"].mean()
    lo_std = city_attributes["Longitude"].std()
    city_attributes["Longitude"] = (city_attributes["Longitude"] - lo_mean) / lo_std

def to_city_day_vect(ctv, wind_treshold):
    cdv = {}
    for city, df in ctv.items():
        u = 0
        cdr = []
        while u < len(df):
            w = u+24
            date = df.iloc[u, 0]
            vec = np.reshape(df.iloc[u:w, 1:].to_numpy(), -1)
            temp_mean = df.iloc[u:w]["temperature"].mean()
            wind_cat = int(np.any(df.iloc[u:w]["wind_speed"].to_numpy() > wind_treshold))
            cdr.append((date, vec, temp_mean, wind_cat))
            u = w
        cdv[city] = cdr
    return cdv

def get_city_encoder(cities_attr):
    cities = np.reshape(city_attributes_raw["City"].to_numpy(), (-1, 1))
    cohe = OneHotEncoder()
    cohe.fit(cities)
    return cohe

def get_wind_treshold(souce, params):
    mean, std = params["wind_speed"]
    return (souce - mean)/std

def drop_nan_records(data_set):
    mask = [np.any(np.isnan(val), axis=0) for val in data_set.values()]
    mask = np.vstack(mask)
    mask = np.any(mask, axis = 0)
    return {key: val[:, ~mask] for key, val in data_set.items()}

def get_set1(cdv, city_encoder, city_attributes_raw):
    d1 = []
    d2 = []
    d3 = []
    output_temp = []
    output_wind = []
    date = []
    city_one_hot = []
    cord = []

    for city, dv in cdv.items():
        d1 += [r[1] for r in dv[:-4]]
        d2 += [r[1] for r in dv[1:-3]]
        d3 += [r[1] for r in dv[2:-2]]
        output_temp += [r[2] for r in dv[4:]]
        output_wind += [r[3] for r in dv[4:]]
        date_str = [r[0] for r in dv[4:]]
        date += [datetime.datetime.strptime(d, "%d.%m.%Y %H:%M").timetuple().tm_yday / 365 - 0.5 for d in date_str]
        size = len(date_str)
        city_one_hot += [city_encoder.transform([[city]]).toarray()[0]]*size
        cord += [city_attributes_raw.loc[city_attributes_raw["City"] == city][["Latitude", "Longitude"]].to_numpy()]*size

    set = {
        "d1": d1,
        "d2": d2,
        "d3": d3,
        "output_temp": output_temp,
        "output_wind": output_wind,
        "date": date,
        "city_one_hot": city_one_hot,
        "cord": cord
    }

    return {key: np.vstack(val).T for key, val in set.items()}

city_attributes_raw = read_csv("data/city_attributes.csv", sep=";")
train_raw = load_train()
nerast_cities = get_nearest_cittes(city_attributes_raw)
normalization_params = get_normalization_params(train_raw)
train_ctv = to_city_time_vect(train_raw)
normalize(train_ctv, normalization_params)
normalize_city_attributes(city_attributes_raw)
wind_treshold = get_wind_treshold(6, normalization_params)
train_cdv = to_city_day_vect(train_ctv, wind_treshold)
city_encoder = get_city_encoder(city_attributes_raw)
train_set = get_set1(train_cdv, city_encoder, city_attributes_raw)
train_set = drop_nan_records(train_set)

test_raw = load_test()
test_ctv = to_city_time_vect(test_raw)
normalize(test_ctv, normalization_params)
test_cdv = to_city_day_vect(test_ctv, wind_treshold)
test_set = get_set1(test_cdv, city_encoder, city_attributes_raw)
test_set = drop_nan_records(test_set)


In [189]:
import neural_network as nn
import definitions as d

rng = np.random.default_rng(0)

def get_day_layer(num):
    l = nn.InputLayer(120, f"d{num}")
    return nn.FullConnectLayer(l, 60, d.relu, rng)

def get_days_layer():
    ls = [get_day_layer(1), get_day_layer(2), get_day_layer(3)]
    l = nn.MergeLayer(ls)
    return nn.FullConnectLayer(l, 60, d.relu, rng)

def get_city_layer():
    coh = nn.InputLayer(36, "city_one_hot")
    date = nn.InputLayer(1, "date")
    cord = nn.InputLayer(2, "cord")
    return nn.MergeLayer([coh, date, cord])

def get_nn():
    ds = get_days_layer()
    c = get_city_layer()
    l = nn.MergeLayer([ds, c])
    l = nn.FullConnectLayer(l, 60, d.relu, rng)
    l = nn.FullConnectLayer(l, 1, d.linear, rng)
    return nn.NeuralNetwork(l, d.l2_loss)

net = get_nn()

In [190]:
net.train(train_set, test_set, 1024, "output_temp", rng)

Train: 0.7304230707869158, test: 0.13082724761831813
Train: 0.11612814257231642, test: 0.1285674178759632
Train: 0.10612802642849753, test: 0.11514731651606276
Train: 0.1007374571051435, test: 0.11213547131728856
Train: 0.09658725857278032, test: 0.11016502489013917
Train: 0.09435004585178133, test: 0.11171731599489414
Train: 0.09188860398254166, test: 0.11308086197756438
Train: 0.09018225385852241, test: 0.11387330817783387
Train: 0.08848884224937081, test: 0.11467544843545381
Train: 0.08697787565571195, test: 0.11441859697945708
Train: 0.08566401267493495, test: 0.12187779343010574
Train: 0.08501676310552572, test: 0.12292089541896146
Train: 0.08360090465821626, test: 0.11566952512062237
Train: 0.08155023997378692, test: 0.11665951612418546
Train: 0.08024747380548478, test: 0.11549110996832024
Train: 0.07937389545252099, test: 0.11107701014485458
Train: 0.0778092205444004, test: 0.11251638588814053
Train: 0.07653225690009707, test: 0.11373908616195283
Train: 0.07499096125571647, test

KeyboardInterrupt: 